# Part 2

### 1. Explain the distance metric you utilized to calculate the similarity/dissimilarity between small molecules.

I used 1 - (jaccard index) as a distance metric. The jaccard index is the intersect of members divided by the union of members, which gives a measure of similarity between the two groups. Here, for two ligands, the JI is calculated by taking the intersect of their on bits divided by the union. Since this is a measure of similarity between 0 and 1, I do 1 - JI to get a measure of difference or distance.

### 2. Use a dimensionality reduction algorithm (PCA, t-SNE, UMAP, etc) to generate a 2D visualization of the small molecule dataset.

In [1]:
from clusters.algs import *
#from scipy.sparse import lil_matrix
#import umap
#import umap.plot
#import matplotlib.pyplot as plt


In [ ]:
# Load data (downsampling to the first 2000 ligands)
table = pd.read_csv("./ligand_information.csv")
table['OnBits'] = table['OnBits'].str.split(",")
table = table.head(n = 2000)

# Create a list of lists of onbits for each ligand
lil = list(table['OnBits'])
lil = [list(map(int, i)) for i in lil]

# Create sparse matrix
factor_matrix = lil_matrix((len(table), 1024), dtype=np.float32)
factor_matrix.rows = np.array(lil)
data = [[1] * len(i) for i in lil]
factor_matrix.data = np.array(data)


In [ ]:
factor_matrix

In [ ]:
# fit umap
mapper = umap.UMAP(metric='cosine', random_state=42, low_memory=True).fit(factor_matrix)


In [ ]:
umap.plot.points(mapper)

### 3. Cluster the small molecules using your implementation of a partitioning clustering algorithm. Visualize this clustering by coloring clusters on the 2D visualization generated in question 2.

In [ ]:
weights = np.ones(len(ligands))
random.choices(range(len(ligands)), weights)[0]

In [ ]:
pc = PartitionClustering()
ligands = pc.get_ligands(10)
pc.cluster(ligands, k=3)

In [6]:
pc = PartitionClustering()
ligands = pc.get_ligands(10)
c = pc.cluster(ligands, k=3)

[2, 0, -1, -1, -1, -1, -1, -1, -1, 1]


In [ ]:
lig = 5 
chosen = [5]
i = 0
while(lig in chosen):
    print("hello")
    if i == 2:
        lig = 6
    i += 1

In [ ]:
k= 3
d = pc.distance_matrix(ligands)
centroids = []
clusters = [-1] * len(ligands)

# start with equal weights for all ligands
weights = np.ones(len(ligands))

for i in range(k):
    lig = random.choices(range(len(ligands)), weights)[0]
    clusters[lig] = i
    centroids.append(ligands[lig].onbits)
    # update weights to the distance between lig and all other ligs
    weights = copy.deepcopy(d[lig,:])
    weights[np.where(weights == float("inf"))] = -10 # downweight the same ligand


In [ ]:
clusters

In [ ]:
centroids

In [ ]:
pc = PartitionClustering()
ligands = pc.get_ligands(len(table))
clusters = {}
sil_scores = []

# Try different values of k 
for i in range(2,5):
    clustering = pc.cluster(ligands, k=i)
    clusters[i] = clustering
    sil_scores.append(pc.silhouette_score(ligands, clustering))

In [ ]:
sil_scores

In [ ]:
clustering = [0] * len(ligands)
for i in range(len(clusters)):
    for j in range(len(clusters[i])):
        idx = clusters[i][j]
        clustering[idx] = i


In [ ]:
umap.plot.points(mapper, np.array(clusters))

In [ ]:
clusters

In [ ]:
clustering

In [ ]:
table['HC'] = clustering_hc
table['PC'] = clustering

In [ ]:
clusters_hc

In [ ]:
table.head(n=100)

In [ ]:
import sklearn.cluster as cluster
kmeans_labels = cluster.KMeans(n_clusters=3).fit_predict(factor_matrix)

In [ ]:
kmeans_labels

In [ ]:
umap.plot.points(mapper, kmeans_labels)

### 4. Explain your choice of partitioning clustering algorithm. Is it sensitive to initialization conditions? How do you select the number of clusters?

In my partition clustering algorithm, the clusters are initialized by randomly choosing k ligands as centroids. Yes, the final clustering is sensitive to initialization and may vary each time it is run due to the random initialization. 


In [ ]:
ligands = pc.get_ligands(20)
clusters_pc = pc.cluster(ligands, k=2)
clusters_pc

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

dist = np.zeros((len(ligands),len(ligands)))
for i in range(len(ligands)):
    for j in range(len(ligands)):
        if i == j:
            continue
        dist[i,j] = hc.calculate_distance(ligands[i].onbits,ligands[j].onbits)



In [ ]:
a = AgglomerativeClustering(affinity="precomputed", n_clusters=2,linkage="single").fit_predict(dist)


In [ ]:
a

In [ ]:
umap.plot.points(mapper, a)

In [ ]:
dist

### 5. Cluster the small molecules using your implementation of a hierarchical clustering algorithm. Visualize this clustering in the same way as question 3.

In [ ]:
hc = HierarchicalClustering()
clusters_hc = hc.cluster(ligands, k=4)

In [ ]:
clustering_hc = [0] * len(ligands)
for i in range(len(clusters_hc)):
    for j in range(len(clusters_hc[i])):
        idx = clusters_hc[i][j]
        clustering_hc[idx] = i


In [ ]:
umap.plot.points(mapper, np.array(clustering_hc))

### 6. Explain your choice of hierarchical clustering algorithm. Is it sensitive to initialization conditions? How do you select the number of clusters?

I used single linkage in my HC algorithm, where the distance between two clusters is the minimum distance between their points. This implementation should not be sensitive to initialization because it calculates a distance matrix to group the ligands, which is not random. 

### 7. Evaluate the quality of both clusterings using your implementation of a clustering quality metric. Explain your choice of quality metric. Which clustering performed ‘best’ according to your metric?

In [ ]:
hc = HierarchicalClustering()
hc.silhouette_score(ligands, clusters_hc)

In [ ]:
pc = PartitionClustering()
clusters_pc = pc.cluster(ligands, k=4)
pc.silhouette_score(ligands, clusters_pc)